In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path

# Direct path specification
env_path = Path('../backend/.env').resolve()  # or Path('../backend/.env') depending on your directory structure

# Load environment variables from .env file with explicit path
load_dotenv(dotenv_path=env_path)

# Access your API key
api_key = os.getenv("EMBER_API_KEY")

# Verify it loaded (don't print the actual key in your notebook!)
print("API key loaded:", "✓" if api_key else "✗")
print(f"Looking for .env file at: {env_path}")

API key loaded: ✓
Looking for .env file at: /Users/allison/workspace/ai playground/steering-interface/backend/.env


In [2]:
import goodfire

In [3]:
client = goodfire.Client(
    api_key=api_key,
)

In [4]:
response =  client.chat.completions.create(
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ],
    model="meta-llama/Llama-3.3-70B-Instruct"
)

print(response.choices[0].message["content"])

The capital of France is Paris.


In [5]:
# Create a variant

vDefault = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")
print(vDefault)

Variant(
   base_model=meta-llama/Llama-3.3-70B-Instruct,
   edits={
   }
   scopes={
   }
)


In [6]:
# Search features

feature = client.features.search("ELI5", model=vDefault, top_k=10)
print(feature)

FeatureGroup([
   0: "Simplified explanations of complex concepts",
   1: "Explaining complex topics in simpler terms to help understanding",
   2: "Explaining complex concepts using simple, accessible language",
   3: "Breaking down technical concepts into simpler explanations",
   4: "Breaking down complex topics into simpler explanations",
   5: "Breaking down complex topics into simpler explanations",
   6: "The assistant is providing a simplified technical explanation",
   7: "The assistant is simplifying a complex technical concept",
   8: "Making complex concepts understandable to others",
   9: "Detailed explanation of inner workings or processes"
])


In [7]:
inspector = client.features.inspect(
    [
        {"role": "user", "content": "What do you think about pirates and whales"},
        {"role": "assistant", "content": "I think pirates are cool and whales are cool"}
    ],
    model=vDefault
)

# Get top activated features
for activation in inspector.top(k=20):
    print(f"{activation.feature.label}: {activation.activation}")

Technical setup and configuration states in experimental procedures: 7
The assistant should engage with pirate-themed content or roleplay as a pirate: 7
The assistant acknowledges its AI limitations while pivoting to what it can offer: 7
Detailed personification and attribute description in formal writing: 7
Setting up detailed writing or description tasks: 5
Explaining that something from fiction is not real or possible: 5
Whales: 5
The start of instructional or procedural steps in lists and guides: 5
The assistant begins a detailed helpful response: 4
The assistant is explaining its nature as an AI system: 4
Transitional phrases that make explanations flow naturally: 4
The user has a question: 3
Polite user requests requiring detailed responses: 3
Opinion-seeking prepositions in evaluation requests: 3
Word parts expressing opinions or viewpoints across different languages: 3
Intelligence in animals: 3
The introduction of explanatory content about benefits or advantages: 3
Prompting f

In [8]:
# Compare formal vs informal conversations
dataset_1 = [[
    {"role": "user", "content": "Hi how are you?"},
    {"role": "assistant", "content": "I'm doing well..."}
]]
dataset_2 = [[
    {"role": "user", "content": "Hi how are you?"},
    {"role": "assistant", "content": "Arr my spirits be high..."}
]]

formal_features, informal_features =  client.features.contrast(
    dataset_1=dataset_1,
    dataset_2=dataset_2,
    model="meta-llama/Llama-3.3-70B-Instruct",
    top_k=5
)

In [12]:
formal_features
random_feature = formal_features[0]

In [10]:
# Rerank features based on relevance to "writing style"
reranked = client.features.rerank(
    features=formal_features,
    query="writing style",
    model="meta-llama/Llama-3.3-70B-Instruct",
    top_k=10
)

reranked

FeatureGroup([
   0: "Assistant responding to casual greetings about its wellbeing",
   1: "Status check questions and responses using 'doing' in conversation",
   2: "The assistant maintaining honesty about being an AI",
   3: "Assistant's initial friendly greeting to open a conversation",
   4: "Positive responses to 'how are you' questions across languages"
])

In [20]:
random_feature.uuid

'8bb53ed7-ed2f-4166-9161-13df6006451d'